In [65]:
from pyspark.sql import SparkSession
session = SparkSession.builder.appName('bigdata').getOrCreate()
df = session.read.csv('dataset/bigdata.csv',header=True,inferSchema= True)
df.show(n=10)

+--------+-----+----+-----+
|    date|store|item|sales|
+--------+-----+----+-----+
|02/01/13|    1|   1|   11|
|02/02/13|    1|   1|   21|
|02/03/13|    1|   1|   15|
|02/04/13|    1|   1|   14|
|02/05/13|    1|   1|    9|
|02/06/13|    1|   1|   10|
|02/07/13|    1|   1|   13|
|02/08/13|    1|   1|   11|
|02/09/13|    1|   1|   14|
|02/10/13|    1|   1|   11|
+--------+-----+----+-----+
only showing top 10 rows



In [ ]:
## count row
# df.count()

In [ ]:
# df.printSchema()

In [ ]:
# df = df.na.drop(how='any')
# df.count()
# col_name = df.columns
# col_name

## we need to modify date field by Replace Column Value Character by Character (/ by -)

By using translate() string function you can replace character by character of DataFrame column value.

In [ ]:
###very important code
# from pyspark.sql.functions import translate
# df = df.withColumn('date', translate('date', '/', '-'))
# df.show(n=10)
#This unix_timestamp support only dd-mm-yy ex (02-12-21). but our row data was some of them this format somr of them
#10/12/2021 format. so at frist I format this to 10/12/2021 format in excel. then I replaced / by - using this line. then 
# I apply from_unixtime funsion and change to 

In [ ]:
#seperate datetime
# from pyspark.sql.functions import year
# from pyspark.sql.functions import to_timestamp,date_format
# from pyspark.sql.functions import col
# from pyspark.sql.functions import unix_timestamp, from_unixtime

# df2 = df.select(
#     'date', 'store', 'item', 'sales', 
#     from_unixtime(unix_timestamp('date', 'dd-mm-yy')).alias('datetime')
# )
# df2.show(n=10)

In [ ]:
# df2.printSchema()

## Separate datetime : day ,month, year 

In [ ]:
# df2 = df2.withColumn('month',month(df2.datetime)).withColumn('year',year(df2.datetime))
# df2.show(10)

In [ ]:
# df2.columns

In [ ]:
# ## Easy way to retrive day

# df2 = df2.select(
#     'date', 'store', 'item', 'sales', 'datetime', 'month', 'year', 
#     dayofmonth("datetime").alias('day')
# )
# df2.show(n=100)

# final_df = df2.select(
#     'date','datetime',
#     year("datetime").alias('year'), 
#     month("datetime").alias('month'),
#     dayofmonth("datetime").alias('day'),
#     'store', 'item', 'sales',
# )
# final_df.show(n=10)

 ## Separate datetime : day ,month, year (effective way)

In [ ]:
# initailize SparkSession and load data

In [66]:
df = df.na.drop(how='any')
df.count()
col_name = df.columns
col_name

['date', 'store', 'item', 'sales']

In [67]:
###very important code
from pyspark.sql.functions import translate,to_date,split
final_df = df.withColumn('date', translate('date', '/', '-'))
#This unix_timestamp support only dd-mm-yy ex (02-12-21). but our row data was some of them this format somr of them
#10/12/2021 format. so at frist I fortamt this to 10/12/2021 format in excel. then I replaced / by - using this line. then 
# I apply from_unixtime funsion and change to 
final_df.show()

+--------+-----+----+-----+
|    date|store|item|sales|
+--------+-----+----+-----+
|02-01-13|    1|   1|   11|
|02-02-13|    1|   1|   21|
|02-03-13|    1|   1|   15|
|02-04-13|    1|   1|   14|
|02-05-13|    1|   1|    9|
|02-06-13|    1|   1|   10|
|02-07-13|    1|   1|   13|
|02-08-13|    1|   1|   11|
|02-09-13|    1|   1|   14|
|02-10-13|    1|   1|   11|
|02-11-13|    1|   1|   16|
|02-12-13|    1|   1|   11|
|13-02-13|    1|   1|   14|
|14-02-13|    1|   1|   10|
|15-02-13|    1|   1|   11|
|16-02-13|    1|   1|    7|
|17-02-13|    1|   1|   11|
|18-02-13|    1|   1|   10|
|19-02-13|    1|   1|   10|
|20-02-13|    1|   1|    7|
+--------+-----+----+-----+
only showing top 20 rows



In [69]:
final_df = final_df.withColumn('day', split(final_df['date'], '-').getItem(0)).withColumn('month', split(final_df['date'], '-').getItem(1)).withColumn('year', split(final_df['date'], '-').getItem(2))
final_df.show(truncate=False)

+--------+-----+----+-----+---+-----+----+
|date    |store|item|sales|day|month|year|
+--------+-----+----+-----+---+-----+----+
|02-01-13|1    |1   |11   |02 |01   |13  |
|02-02-13|1    |1   |21   |02 |02   |13  |
|02-03-13|1    |1   |15   |02 |03   |13  |
|02-04-13|1    |1   |14   |02 |04   |13  |
|02-05-13|1    |1   |9    |02 |05   |13  |
|02-06-13|1    |1   |10   |02 |06   |13  |
|02-07-13|1    |1   |13   |02 |07   |13  |
|02-08-13|1    |1   |11   |02 |08   |13  |
|02-09-13|1    |1   |14   |02 |09   |13  |
|02-10-13|1    |1   |11   |02 |10   |13  |
|02-11-13|1    |1   |16   |02 |11   |13  |
|02-12-13|1    |1   |11   |02 |12   |13  |
|13-02-13|1    |1   |14   |13 |02   |13  |
|14-02-13|1    |1   |10   |14 |02   |13  |
|15-02-13|1    |1   |11   |15 |02   |13  |
|16-02-13|1    |1   |7    |16 |02   |13  |
|17-02-13|1    |1   |11   |17 |02   |13  |
|18-02-13|1    |1   |10   |18 |02   |13  |
|19-02-13|1    |1   |10   |19 |02   |13  |
|20-02-13|1    |1   |7    |20 |02   |13  |
+--------+-

In [71]:
final_df.count()
final_df.columns

['date', 'store', 'item', 'sales', 'day', 'month', 'year']

In [73]:
### export data to csv
final_df = final_df.select(['date','day', 'month', 'year', 'store', 'item', 'sales'])
final_df.toPandas().to_csv('dataset/modyfiedBigdata.csv')


+--------+---+-----+----+-----+----+-----+
|    date|day|month|year|store|item|sales|
+--------+---+-----+----+-----+----+-----+
|02-01-13| 02|   01|  13|    1|   1|   11|
|02-02-13| 02|   02|  13|    1|   1|   21|
|02-03-13| 02|   03|  13|    1|   1|   15|
|02-04-13| 02|   04|  13|    1|   1|   14|
|02-05-13| 02|   05|  13|    1|   1|    9|
|02-06-13| 02|   06|  13|    1|   1|   10|
|02-07-13| 02|   07|  13|    1|   1|   13|
|02-08-13| 02|   08|  13|    1|   1|   11|
|02-09-13| 02|   09|  13|    1|   1|   14|
|02-10-13| 02|   10|  13|    1|   1|   11|
|02-11-13| 02|   11|  13|    1|   1|   16|
|02-12-13| 02|   12|  13|    1|   1|   11|
|13-02-13| 13|   02|  13|    1|   1|   14|
|14-02-13| 14|   02|  13|    1|   1|   10|
|15-02-13| 15|   02|  13|    1|   1|   11|
|16-02-13| 16|   02|  13|    1|   1|    7|
|17-02-13| 17|   02|  13|    1|   1|   11|
|18-02-13| 18|   02|  13|    1|   1|   10|
|19-02-13| 19|   02|  13|    1|   1|   10|
|20-02-13| 20|   02|  13|    1|   1|    7|
|21-02-13| 